In [1]:
import pandas as pd
import numpy as np
import sys
sys.path.insert(0,'Utils')
import ArtistExtractor as AE
from ast import literal_eval
import moreFunction

In [24]:
pd.read_csv("TEST_Pipe.csv")

,Unnamed: 0,place,src,address,location,canton,event,date,artists,genre,coordinates
0,0,test,club,address,location_test,NE,simsalabim,2002-22-02,"['Joachim Garraud', '**David Guetta**', 'Pante...",Classic,"(1.0,1.0)"
1,1,test2,club,address,location_test,VD,simsalabim,2002-22-02,"['02:20 - Rihanna', "" Drum'n'bass"", 'David Gue...",Pop,"(1.0,1.0)"
2,2,test3,club,address,location_test,VD,ahahahah,1092-11-11,"['Hanzrudolfmerz', 'Bobdokland', 'ParisLatino'...","['Classic','Rock']","(1.0,1.0)"
3,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
columns = ["place","src","address","location","canton","event","date","artists","genre","coordinates"]
a = pd.DataFrame(columns=[columns])
lineup1 = ["Joachim Garraud","**David Guetta**","Panteros666 ✱ ✲","Rihanna (Live)"]
lineup2 = ["02:20 - Rihanna", " Drum'n'bass", "David Guetta"]
lineup3 = ["Hanzrudolfmerz","Bobdokland","ParisLatino","Rihanna"]
lineup4 = ["Bob Dylan"]

a.loc[0] = ["test","club","address","location_test","NE","simsalabim","2002-22-02",str(lineup1),"Classic","(1.0,1.0)"]
a.loc[1] = ["test2","club","address","location_test","VD","simsalabim","2002-22-02",str(lineup2),"Pop","(1.0,1.0)"]
a.loc[2] = ["test3","club","address","location_test","VD","ahahahah","1092-11-11",str(lineup3),"['Classic','Rock']","(1.0,1.0)"]
a.loc[3] = ["test3","club","address","location_test","VD","ahahahah","1092-11-11",str(lineup4),None,"(1.0,1.0)"]
a.loc[3] = [None,None,None,None,None,None,None,None,None,None]

a.to_csv("TEST_Pipe.csv")
a = pd.read_csv("TEST_Pipe.csv",index_col=0)
events = a
events

,place,src,address,location,canton,event,date,artists,genre,coordinates
0,test,club,address,location_test,NE,simsalabim,2002-22-02,"['Joachim Garraud', '**David Guetta**', 'Pante...",Classic,"(1.0,1.0)"
1,test2,club,address,location_test,VD,simsalabim,2002-22-02,"['02:20 - Rihanna', "" Drum'n'bass"", 'David Gue...",Pop,"(1.0,1.0)"
2,test3,club,address,location_test,VD,ahahahah,1092-11-11,"['Hanzrudolfmerz', 'Bobdokland', 'ParisLatino'...","['Classic','Rock']","(1.0,1.0)"
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
def getArtistsList(df):
    df= df.fillna("None")
    print("Getting artists list from "+str(df.shape[0])+" events...")
    ArtistsDataFrame = pd.DataFrame()
    i=0
    for index,row in df.iterrows():
        lineup = row["artists"] #get line-up
        artists = literal_eval(lineup)
        
        new_lineup=[]
        if(artists!=None):
            for a in artists:
                a = a.split("(")[0]#Remove parenthesis at end of artist (record label, live act..)

                #Use list for convenient format in clean_artists method
                artist_list = []
                artist_list.append(a)
                cleaned_artist = moreFunction.clean_artists(artist_list)
                if(len(cleaned_artist)>0):
                    a = cleaned_artist[0]

                #Append artist in lineup
                new_lineup.append(a.strip())

                #Add artist to artists dataframe
                a = pd.Series(a.strip())
                ArtistsDataFrame = ArtistsDataFrame.append(a,ignore_index=True)

        
        #Replace artist in dataframe
        df.loc[index,"artists"] = str(new_lineup)
        
        #count iteration
        i+=1
        if(i%5000==0):
            print(i)
    
    ArtistsDataFrame.columns=["artist"]
    #ArtistsDataFrame.to_csv("ClubDataTest/ArtistsList.csv",encoding="utf-8")
    #ArtistsDataFrame.head(10)
    ArtistsDataFrame.drop_duplicates()
    return ArtistsDataFrame,df

In [27]:
artists,df = getArtistsList(a)
df

Getting artists list from 4 events...


,place,src,address,location,canton,event,date,artists,genre,coordinates
0,test,club,address,location_test,NE,simsalabim,2002-22-02,"['JoachimGarraud', 'DavidGuetta', 'Panteros666...",Classic,"(1.0,1.0)"
1,test2,club,address,location_test,VD,simsalabim,2002-22-02,"['-Rihanna', ""Drum'n'bass"", 'DavidGuetta']",Pop,"(1.0,1.0)"
2,test3,club,address,location_test,VD,ahahahah,1092-11-11,"['Hanzrudolfmerz', 'Bobdokland', 'ParisLatino'...","['Classic','Rock']","(1.0,1.0)"
3,None,None,None,None,None,None,None,[],None,None


In [22]:
def downloadGenresSpotify(artistsDF,dictionarySpotify=None,begin=0,end=100000):

    Artists = artistsDF.copy()
    Artists = Artists.drop_duplicates().reset_index()[["artist"]]
    Artists.columns=["artist"]
    Artists["genres_spotify"]=None
    Artists["genres_ra"]=None
    Artists["genres_wiki"]=None
    Artists["genres_events"]=None
    Artists["main_genres"]=None
    Artists["top3_genres"]=None
    Artists["genre"]=None    
    
    if(end>Artists.shape[0]):
        end = Artists.shape[0]
        
    if(dictionarySpotify==None):
            dictionarySpotify = {}
    
    print("Downloading genres of "+str(end-begin)+" artist from Spotify...")
    last_read = 0
    for i in range(begin,end):
        S = Artists[Artists.index==i]
        artist = S["artist"].values[0]
        genres_spotify=None
        if(dictionarySpotify!=None and artist in dictionarySpotify):
            genres_spotify = dictionarySpotify.get(artist)
        else:
            genres_spotify = AE.getGenresFromSpotify(artist)
            dictionarySpotify.update({artist : genres_spotify})
            
        
        if(len(genres_spotify)<1):
            genres_spotify = None
        Artists.loc[Artists.index==i,"genres_spotify"] = str(genres_spotify)

    return Artists

In [23]:
artists = downloadGenresSpotify(artists)
artists

,artist,genres_spotify,genres_ra,genres_wiki,genres_events,main_genres,top3_genres,genre
0,JoachimGarraud,None,None,None,None,None,None,None
1,DavidGuetta,None,None,None,None,None,None,None
2,Panteros666?,['destroy techno'],None,None,None,None,None,None
3,Rihanna,"['dance pop', 'pop', 'pop christmas', 'r&b', '...",None,None,None,None,None,None
4,-Rihanna,None,None,None,None,None,None,None
5,Drum'n'bass,None,None,None,None,None,None,None
6,Hanzrudolfmerz,None,None,None,None,None,None,None
7,Bobdokland,None,None,None,None,None,None,None
8,ParisLatino,None,None,None,None,None,None,None


In [8]:
def createDictionnaryFromArtists(artists):
    print("Creating dictionnary from genres downloaded from Spotify...")
    dic = AE.createDictionnary()
    genres_list = []
    for id,row in artists.iterrows():
        genres = literal_eval(row.genres_spotify)
        if(genres!=None):
            genres_list+=(genres)
        
    #print(genres_list)
    dic = AE.updateDictionnary(genres_list,dic)
    return dic                

In [9]:
def downloadGenresWikipediaAndRA(Artists,dictionaryOfGenres,dictionaryWiki=None, dictionaryRA=None, begin=0,end=100000):

    if(end>Artists.shape[0]):
        end = Artists.shape[0]
    
    #path = "FullData/ArtistDataframe_"+str(begin)+"_"+str(end)+".csv"
    print("Downloading genres from Wikipedia and Resident Advisor of "+str(end-begin)+" artist...")
    last_read = 0
    
    if(dictionaryWiki==None):
        dictionaryWiki = {}
    if(dictionaryRA==None):
        dictionaryRA = {}
    
    for i in range(begin,end):
        S = Artists[Artists.index==i]
        artist = S["artist"].values[0]
        
        #Get genres from wikipedia
        genres_wiki = None
        if(dictionaryWiki!=None and artist in dictionaryWiki):
            genres_wiki = dictionaryWiki.get(artist)
        else:
            genres_wiki = AE.getGenresFromWikipedia(artist,dictionaryOfGenres)
            dictionaryWiki.update({artist : genres_wiki})
            
            
        #Get genres from Resident Advisor
        genres_ra = None
        if(dictionaryRA!= None and artist in dictionaryRA):
            genres_ra = dictionaryRA.get(artist)
        else:
            genres_ra = AE.getGenresFromRA(artist,dictionaryOfGenres)
            dictionaryRA.update({artist : genres_ra})
            
        if(genres_wiki== None or len(genres_wiki)<1):
            genres_wiki = None
        if(genres_ra == None or len(genres_ra)<1):
            genres_ra = None
            
        Artists.loc[Artists.index==i,"genres_wiki"] = str(genres_wiki)
        Artists.loc[Artists.index==i,"genres_ra"] = str(genres_ra)

        if(i%300==0):
            print(str(i+1))
            
    return Artists

In [10]:
def computeGenresRatio(artists_df,dictionnaryOfGenres,debug=False):
    genres = list(set(dictionnaryOfGenres.values()))
    genres.append("unknown")
    
    #Extending
    #Getting columns
    columnsArtists = []
    for c in artists_df.columns:
        if("Unnamed" not in c):
            columnsArtists.append(c)
    columnsArtists = columnsArtists+genres+["total_genres"]
    print("Computing genres ratio...")
    i=0
    for id,row in artists_df.iterrows():
        i+=1
        if(debug and i%3000==0):
            print(i)
        main_genres = row["main_genres"]
        
        
        total = 0.0
        
        if(main_genres!=None and main_genres!="None"):
            total = len(literal_eval(main_genres))
            
            #Check if artist has a genre (from other dataframes)
            if(total==0):
                if(raw["genre"]!=None and raw["genre"]!="None"):
                    main_genres = list(literal_eval(raw["genre"]))
                    artists_df.loc[id,"main_genres"] = main_genres
                    total = 1.0
                
        artists_df.loc[id,"total_genres"]=total
        
        #percententage of this genre
        for c in genres:
            if(total==0):
                artists_df.loc[id,c] = 0.0
                artists_df.loc[id,"unknown"]=1.0
            else:
                artists_df.loc[id,c] = main_genres.count(c)/total

    return artists_df

In [11]:
def computeMainGenres(artists,dictionnary):
    
    
    artists["all_genres"] = None
    #dictionnaryOfGenres = LU.loadDictionary("FullData/AllGenresDic",enc="utf-8")
    i=0
    for id,row in artists.iterrows():
        i+=1
        #if(i>20):
         #   break
        if(i%1000==0):
            print(i)

        artist = row["artist"]
        genres_ra = literal_eval(row.genres_ra)
        genres_s = literal_eval(row.genres_spotify)
        genres_w = literal_eval(row.genres_wiki)
        #events
        genres_e = str(row.genres_events)
        genres_e = literal_eval(genres_e)
       
        if(genres_ra==None):
            genres_ra =[]
        if(genres_s == None):
            genres_s = []
        if(genres_w == None):
            genres_w = []
        if(genres_e == None):
            genres_e = []
            
        #Set genres_events to None if there already exist some genres
        if(len(genres_ra)>0 or len(genres_s)>0 or len(genres_w)>0):
            genres_e = []
        
        genres = genres_s+genres_ra+genres_w+genres_e
        
        main_genres = AE.mainGenres(genres,dictionnary)
        top3 = AE.getMaxGenre(main_genres,3)
        main_genre = None

        if(top3!=None and len(top3)>=1):
            main_genre = top3[0]
            
        #Assigning None values for empty lists
        if(len(main_genres)<1):
            main_genres = None
        if(len(genres_ra)<1):
            genres_ra = None
        if(len(genres)<1):
            genres = None
        if(len(genres_e)<1):
            genres_e = None
            
        artists.loc[id,"genres_ra"] = str(genres_ra)
        #artists.loc[id,"genres_events"] = str(genres_e)
        artists.loc[id,"main_genres"] = str(main_genres)
        artists.loc[id,"top3_genres"] = str(top3)
        artists.loc[id,"all_genres"] = str(genres)
        artists.loc[id,"genre"] = main_genre


    #df3.to_csv("FullData/ArtistDF_withGenres.csv",encoding="utf-8")
    return artists

In [12]:
def fillGenresFromEvents(artists,events):
    print("Filling missing genres artists <- events")
  
    i =0
    for id, row in events.iterrows():
        i+=1
        if(i%1000==0):
            print(i)
        
        #getting genre
        str_genres = str(row["genre"])
        genres = []
        
        #case list
        if("[" in str_genres): 
            genres = literal_eval(str_genres)
        #case alone
        else:
            if(str_genres!="None" and str_genres!="nan"):
                genres.append(str_genres)
        
        #reformating
        genres_refo = []
        if(len(genres)>0):
            for g in genres:
                genres_refo.append(g.lower())
            genres = genres_refo
        else:
            genres=None
        #getting artist
        artists_lineup= str(row["artists"])
        
        lineup=[]
        if(artists_lineup!="nan" and artists_lineup!="None" and artists_lineup!=None):
            lineup = (literal_eval(artists_lineup))
        
        for a in lineup:
            artist = artists[artists.artist==a]
            genres_events = artist.genres_events.values
            if(len(genres_events)>0):
                genres_events = genres_events[0]
            else:
                genres_events = None
                
            genres_list = []
            if(genres_events!=None and "None" not in genres_events):
                genres_list+=literal_eval(str(genres_events))
            
            if(genres!=None):
                genres_list+=genres
            if(len(genres_list)<1):
                genres_list=None
                
            artists.loc[artists.artist==a,"genres_events"] = str(genres_list)
            
    return artists

In [13]:
def fillGenresEvents(events,artists,dictionnaryOfGenres):
    print("Filling missing genres events <- artists")
  
    i =0
    for id, row in events.iterrows():
        i+=1
        if(i%1000==0):
            print(i)
        
        lineup = str(row.artists)
        if(lineup!="nan" and lineup!="None" and lineup!=None):
            lineup = literal_eval(lineup)
        else:
            lineup = []
            
        genres_in_lineup = []
        
        #Adding each genre of artist to list genres_in_lineup
        for a in lineup:
            #Getting value
            artist = artists[artists.artist == a]
            genre_artist = None
            if(len(artist.genre.values)>0):
                genre_artist = artist.genre.values[0]
            
            #Add to list of genres   
            if(genre_artist!="nan" and genre_artist!="None" and genre_artist!=None):
                genres_in_lineup.append(genre_artist)
            
            
        #getting most representative genre among all artists
        maxGenre = AE.getMaxGenre(genres_in_lineup)
        print(maxGenre)
        #If an artist has a genre, then we set the genre of event to this artist genre
        if(maxGenre!=None):
            events.loc[id,"genre"] = maxGenre[0].lower()
        else:
            events.loc[id,"genre"] = None
    
    return events,artists

In [14]:
def artistsPipeline(Events,dictionaryOfGenres=None,dictionarySpotify=None,dictionaryRA=None,dictionaryWiki=None):
    if(dictionarySpotify==None):
        dictionarySpotify={}
    if(dictionaryRA ==None):
        dictionaryRA = {}
    if(dictionaryWiki ==None):
        dictionaryWiki = {}
    
    Artists,Events = getArtistsList(Events)
    Artists = downloadGenresSpotify(Artists,dictionarySpotify) #Assign first styles
    
    #Creating the dictionary from artists
    dicGenresArtists = createDictionnaryFromArtists(Artists)
    if(dictionaryOfGenres==None):
        dictionaryOfGenres = dicGenresArtists
    else:
        dictionaryOfGenres.update(dicGenresArtists)
    
    #Getting genres from Wiki And RA
    Artists = downloadGenresWikipediaAndRA(Artists,dictionaryOfGenres,dictionaryWiki,dictionaryRA)
    #Getting genres from Events
    Artists = fillGenresFromEvents(Artists,Events)
    
    #Compute extended fields
    Artists = computeMainGenres(Artists,dictionaryOfGenres)
    Artists = computeGenresRatio(Artists,dictionaryOfGenres)
    Events,Artists = fillGenresEvents(Events,Artists,dictionaryOfGenres)
    print("Computation of genres finished.")
    
    return Artists,Events,dictionaryOfGenres,dictionarySpotify,dictionaryRA,dictionaryWiki

In [15]:
artists

,artist,genres_spotify,genres_ra,genres_wiki,genres_events,main_genres,top3_genres,genre
0,JoachimGarraud,None,None,None,None,None,None,None
1,DavidGuetta,None,None,None,None,None,None,None
2,Panteros666?,['destroy techno'],None,None,None,None,None,None
3,Rihanna,"['dance pop', 'pop', 'pop christmas', 'r&b', '...",None,None,None,None,None,None
4,-Rihanna,None,None,None,None,None,None,None
5,Drum'n'bass,None,None,None,None,None,None,None
6,Hanzrudolfmerz,None,None,None,None,None,None,None
7,Bobdokland,None,None,None,None,None,None,None
8,ParisLatino,None,None,None,None,None,None,None


In [16]:
events

,place,src,address,location,canton,event,date,artists,genre,coordinates
0,test,club,address,location_test,NE,simsalabim,2002-22-02,"['Joachim Garraud', '**David Guetta**', 'Pante...",Classic,"(1.0,1.0)"
1,test2,club,address,location_test,VD,simsalabim,2002-22-02,"['02:20 - Rihanna', "" Drum'n'bass"", 'David Gue...",Pop,"(1.0,1.0)"
2,test3,club,address,location_test,VD,ahahahah,1092-11-11,"['Hanzrudolfmerz', 'Bobdokland', 'ParisLatino'...","['Classic','Rock']","(1.0,1.0)"
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
artists,events,genres,spotify,ra,wiki = artistsPipeline(events)

Getting artists list from 4 events...
Creating dictionnary from genres downloaded from Spotify...
1
Filling missing genres artists <- events
Computing genres ratio...
Filling missing genres events <- artists
['electronic']
['electronic']
['orchestral']
None
Computation of genres finished.


In [18]:
artists

,artist,genres_spotify,genres_ra,genres_wiki,genres_events,main_genres,top3_genres,genre,all_genres,total_genres,...,latin,r&b and soul,jazz,hip hop,unknown,avant-garde,electronic,blues,comedy,african
0,JoachimGarraud,None,None,None,['classic'],['orchestral'],['orchestral'],orchestral,['classic'],1.0,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0
1,DavidGuetta,None,"['techno', 'rock', 'electropop', 'coldwave', '...",None,"['classic', 'pop']","['electronic', 'rock', 'pop', 'electronic', 'p...","['electronic', 'pop', 'hip hop']",electronic,"['techno', 'rock', 'electropop', 'coldwave', '...",7.0,...,0.000000,0.000000,0.000000,0.142857,0.0,0.0,0.285714,0.0,0.0,0.0
2,Panteros666?,['destroy techno'],None,None,['classic'],['electronic'],['electronic'],electronic,['destroy techno'],1.0,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,1.000000,0.0,0.0,0.0
3,Rihanna,"['dance pop', 'pop', 'pop christmas', 'r&b', '...",None,"['rock', 'soca', 'rap', 'chanson', 'dancehall'...","['classic', 'classic', 'rock']","['pop', 'pop', 'pop', 'r&b and soul', 'r&b and...","['pop', 'r&b and soul', 'caribbean and caribbe...",pop,"['dance pop', 'pop', 'pop christmas', 'r&b', '...",19.0,...,0.052632,0.210526,0.000000,0.105263,0.0,0.0,0.105263,0.0,0.0,0.0
4,-Rihanna,None,None,None,['pop'],['pop'],['pop'],pop,['pop'],1.0,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0
5,Drum'n'bass,None,None,"['doomcore', 'hardstyle', 'darkcore', 'dub', '...",['pop'],"['electronic', 'electronic', 'electronic', 'el...","['electronic', 'hip hop', 'r&b and soul']",electronic,"['doomcore', 'hardstyle', 'darkcore', 'dub', '...",35.0,...,0.000000,0.057143,0.028571,0.114286,0.0,0.0,0.685714,0.0,0.0,0.0
6,Hanzrudolfmerz,None,None,None,"['classic', 'rock']","['orchestral', 'rock']","['orchestral', 'rock']",orchestral,"['classic', 'rock']",2.0,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0
7,Bobdokland,None,None,None,"['classic', 'rock']","['orchestral', 'rock']","['orchestral', 'rock']",orchestral,"['classic', 'rock']",2.0,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0
8,ParisLatino,None,None,None,"['classic', 'rock']","['orchestral', 'rock']","['orchestral', 'rock']",orchestral,"['classic', 'rock']",2.0,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0


In [19]:
events

,place,src,address,location,canton,event,date,artists,genre,coordinates
0,test,club,address,location_test,NE,simsalabim,2002-22-02,"['JoachimGarraud', 'DavidGuetta', 'Panteros666...",electronic,"(1.0,1.0)"
1,test2,club,address,location_test,VD,simsalabim,2002-22-02,"['-Rihanna', ""Drum'n'bass"", 'DavidGuetta']",electronic,"(1.0,1.0)"
2,test3,club,address,location_test,VD,ahahahah,1092-11-11,"['Hanzrudolfmerz', 'Bobdokland', 'ParisLatino'...",orchestral,"(1.0,1.0)"
3,None,None,None,None,None,None,None,[],None,None
